<a href="https://colab.research.google.com/github/derick1castro/Previsao-de-vendas/blob/main/Previsao_de_vendas_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1° Entendimento do Problema de Negócio


A empresa solicitou a equipe de Ciência de Dados que desenvolvessem uma metodologia Preditiva que utilizasse IA com o objetivo de encontrar, entre seus Leads, quais são aqueles que estão mais propensos a comprarem seu produto, dada uma campanha de Marketing.

Ou seja, a empresa quer saber se, o cliente fosse atingido por uma campanha de marketing, qual seria o resultado:

O cliente compra

ou

O cliente não compra

#2° Análise Exploratória dos Dados

In [ ]:
# Biblioteca Python para Análise de Dados
!pip install pandas-profiling

In [ ]:
# Biblioteca Python para Realizar o acesso aos Dados
!pip install gdown

In [11]:
#Bibliotecas a serem utilizadas
import gdown
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

Caminho para o download dos dados utilizados para a análise dos Dados.

In [ ]:
# URL do arquivo do Google Drive
url = 'https://drive.google.com/file/d/1_kq8cZ4JZVsYw0BamYfiohSngLIJkoVb/view?usp=sharing'
output = 'dataset.csv'  # nome do arquivo que será salvo localmente

# Transformando o link compartilhado em um link direto para download
file_id = url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id

# Fazendo o download do arquivo
gdown.download(dwn_url, output, quiet=False)

In [5]:
# Carregando o arquivo CSV com o pandas
df = pd.read_csv(output, delimiter=',')

# Mostrando as primeiras linhas do dataframe
df.head()

,idade,emprego,estado_civil,educacao,padrao,saldo,moradia,emprestimo,contato,dia,mes,duracao,campanha,dias_passados,anterior,resultado_anterior,prevendas
0,58,gerente,casado,terciário,não,2143,sim,não,desconhecido,5,maio,261,1,-1,0,desconhecido,não
1,44,técnico,solteiro,secundário,não,29,sim,não,desconhecido,5,maio,151,1,-1,0,desconhecido,não
2,33,empreendedor,casado,secundário,não,2,sim,sim,desconhecido,5,maio,76,1,-1,0,desconhecido,não
3,47,trabalhador-manual,casado,desconhecido,não,1506,sim,não,desconhecido,5,maio,92,1,-1,0,desconhecido,não
4,33,desconhecido,solteiro,desconhecido,não,1,não,não,desconhecido,5,maio,198,1,-1,0,desconhecido,não


In [ ]:
#Visualização do Relatório na WEB
from IPython.display import display, HTML

# Crie o relatório de perfil usando o Pandas Profiling
profile = ProfileReport(df)

# Salve o relatório em HTML
profile.to_file("output.html")

# Exiba o relatório de perfil
display(HTML(profile.to_html()))

# 3° Processamento de Dados

### Importação das bibliotecas

In [8]:
# Bibliotecas de IA - Machine Learning e Automações
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle

### Amostragem dos dados

In [9]:
# Separar features e target
X = df.drop('prevendas', axis=1)
y = df['prevendas']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Engenharia de Atributos

In [ ]:
# Listar as colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Criar os transformers para pré-processamento
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse=False)

# Criar o ColumnTransformer para aplicar as transformações nas colunas apropriadas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Aplicar as transformações no preprocessor
X_train_preprocessed = preprocessor.fit_transform(X_train)
numeric_columns = numeric_features.tolist()
categorical_columns = preprocessor.transformers_[1][1].get_feature_names_out(categorical_features)
all_columns = numeric_columns + list(categorical_columns)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=all_columns)



In [13]:
# Cria Dados Sintéticos para resolver o problema de Desbalanceamento do Target
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_preprocessed, y_train)


#4° Construção e Avaliação da IA Preditiva de Vendas

### Escolha do Algoritmo de Machine Learning e Treinamento com os Dados

In [14]:
# Escolher um algoritmo de aprendizado de máquina
model = LGBMClassifier(n_estimators=200)

# Treinar o modelo no conjunto de treinamento
pipeline = make_pipeline(model)
pipeline.fit(X_train_resampled, y_train_resampled)

# Aplicar as mesmas transformações no conjunto de teste
X_test_preprocessed = preprocessor.transform(X_test)
X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=all_columns)

### Avaliação da IA com as Métricas

In [15]:
# Avaliar o modelo no conjunto de teste
y_pred = pipeline.predict(X_test_preprocessed)

# Métricas de avaliação
print('Acurácia:', accuracy_score(y_test, y_pred))
print('\nClassification Report:\n', classification_report(y_test, y_pred))

Acurácia: 0.910206789782152

Classification Report:
               precision    recall  f1-score   support

         não       0.95      0.95      0.95      7952
         sim       0.64      0.60      0.62      1091

    accuracy                           0.91      9043
   macro avg       0.79      0.77      0.78      9043
weighted avg       0.91      0.91      0.91      9043



### Lista das Variáveis mais Importantes da IA

In [16]:
import plotly.express as px
import plotly.graph_objects as go

# Obter a importância das variáveis
importance = pipeline.named_steps['lgbmclassifier'].feature_importances_
feature_names = all_columns

# Criar um DataFrame com a importância das variáveis
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})

# Ordenar o DataFrame pela importância das variáveis em ordem decrescente
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Criar o gráfico de barras da importância das variáveis usando o Plotly
fig = go.Figure(data=[go.Bar(x=importance_df['Importance'], y=importance_df['Feature'], orientation='h')])
fig.update_layout(
    title='Importância das Variáveis',
    xaxis_title='Importância',
    yaxis_title='Variáveis'
)
# Inverta a ordem do eixo y para que o recurso mais importante esteja no topo
fig.update_yaxes(autorange="reversed")

fig.show()

### Salvamento da IA

In [17]:
# Salvar o modelo treinado, o pré-processador e os nomes das colunas em um arquivo
with open('model.pkl', 'wb') as f:
    pickle.dump({
        'model': pipeline,
        'preprocessor': preprocessor,
        'columns': all_columns
    }, f)